# **Importamos la librerias necesarias**

In [ ]:
from _utils import pd, read_json, find_columns_with_nulls

# **Ahora cargamos los dataset necesarios**

In [ ]:
df = pd.DataFrame(read_json("../../datasets/json/coins.json"))

In [ ]:
type(df)

# **Inspeccion de Datos**


Ahora le damos una vista general de los datos y se limpiaran los datos 

`df_coins`

In [ ]:
df.head(5)

In [ ]:
df.info()

Observamos la frecuencia de las exchanges que pertenece cada moneda

In [ ]:
# Cantidad de exchanges centralizados (frecuencia)
df["exchanges_centralized"].apply(lambda x: len(x)).value_counts()[:10]

In [ ]:
# Cantidad de exchanges descentralizados (frecuencia)
df["exchanges_decentralized"].apply(lambda x: len(x)).value_counts()[:10]

## Caso especial

Antes de realizar la limpieza de los datos, podemos observar los siguiente del `JSON` del dataset:
- La exchange a la cual pertenece es una version de esta pasada `uniswap_v2`, ya que actualmente esta la version 3.
- La criptomoneda no ha tenido movimiento las ultimas 24 horas.
- No tiene maximos ni minimos historicos.
  
Por lo cual podemos deducir que se debe a un error en el registro de la moneda o un problema con la version de la exchange de donde la API nos proporciono esta información 

```json
{
	"id": "party",
	"symbol": "party",
	"name": "PARTY",
	"image": "https://assets.coingecko.com/coins/images/33550/large/party-logo-200x200.png?1702397278",
	"current_price": null,
	"market_cap": null,
	"market_cap_rank": null,
	"fully_diluted_valuation": null,
	"total_volume": null,
	"high_24h": null,
	"low_24h": null,
	"price_change_24h": null,
	"price_change_percentage_24h": null,
	"market_cap_change_24h": null,
	"market_cap_change_percentage_24h": null,
	"circulating_supply": null,
	"total_supply": 420177600000000.0,
	"max_supply": 420177600000000.0,
	"ath": null,
	"ath_change_percentage": 0.0,
	"ath_date": null,
	"atl": null,
	"atl_change_percentage": 0.0,
	"atl_date": null,
	"roi": null,
	"last_updated": null,
	"category": "Pixels Game",
	"category_id": "pixels-game",
	"exchanges_centralized": [],
	"exchanges_decentralized": [
		"uniswap_v2"
	]
}
```

Este caso especial se tendra que eliminar del dataset 

In [ ]:
# Eliminar la fila con id "party"
df = df[df["id"] != "party"]

In [ ]:
# Validamos que no existe ese json
df[df["id"] == "party"]

# **Preprocesamiento de los Datos**

Eliminamos primero la columna `roi`, ya que no nos proporciona informacion relevante para el analisis

In [ ]:
# Eliminamos la columna `roi`
df.drop(columns=["roi"], inplace=True)

Ahora verificamos que columnas tienen valores nulos

In [ ]:
find_columns_with_nulls(df)

Eliminamos el unico registro que no tiene `current_price`

In [ ]:
df = df[~df["current_price"].isnull()]

## Ahora revisamos la columna de `market_cap_rank`

In [ ]:
find_columns_with_nulls(df)[0]

Observamos el maximo valor de `market_cap_rank`

In [ ]:
df["market_cap_rank"].max() 

In [ ]:
df[3622:].head(4)

Observamos que solo hay ranking hasta el numero `4333`, asi que rellenamos los nulos de esta columna con un `-1`

In [ ]:
df["market_cap_rank"].fillna(-1, inplace=True) # Rellenar con -1
df["market_cap_rank"] = df["market_cap_rank"].astype(int) # Convertir a entero

## Ahora revisamos la columna `fully_diluted_valuation`

In [ ]:
find_columns_with_nulls(df)[0]

Observamos a detalle y notamos que:
- `fully_diluted_valuation` es igual a `total_supply * current_price` en ciertos casos
- Cuando `total_supply` es nulo, `fully_diluted_valuation` tambien es nulo


In [ ]:
# Monedas con valor de `fully_diluted_valuation` igual al valor de `total_supply` por `current_price`
df[df["fully_diluted_valuation"] == df["total_supply"]*df["current_price"]].iloc[:3, 4:20]

In [ ]:
df[df["fully_diluted_valuation"].isnull()].iloc[:3, 7:20]

Entonces rellenaremos con el valor de `total_supply * current_price` cuando `fully_diluted_valuation` sea nulo y `total_supply` no sea nulo

In [ ]:
df["fully_diluted_valuation"].fillna(df["total_supply"]*df["current_price"], inplace=True)

In [ ]:
find_columns_with_nulls(df)[0]

Ahora los restantes que no tenemos datos rellenaremos con -1

In [ ]:
df["fully_diluted_valuation"].fillna(-1, inplace=True) # Rellenar con -1

## Ahora verificamos la columna `high_24h`


In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
df[df["high_24h"].isnull()].iloc[:5, 0:20]

Observamos que en otros registros el high_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`

In [ ]:
df[df["current_price"] == df["high_24h"]][:5]

In [ ]:
df["high_24h"].fillna(df["current_price"], inplace=True)

## Verificamos la columna `low_24h`

In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
df[df["low_24h"].isnull()].iloc[:5, 0:20]

Observamos que en otros registros el low_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`


In [ ]:
df[df["current_price"] == df["low_24h"]][:5]

In [ ]:
df["low_24h"].fillna(df["current_price"], inplace=True)

## Verificamos la columna `price_change_24h` y `price_change_percentage_24h`

In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
df[df["price_change_24h"].isnull()].iloc[:5, 0:20]

In [ ]:
df["price_change_24h"].fillna(0, inplace=True)
df["price_change_percentage_24h"].fillna(0, inplace=True)

## Verificamos la columna `market_cap_change_24h` y `market_cap_change_percentage_24h`


In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
find_columns_with_nulls(df)[1]

In [ ]:
df[df["market_cap_change_24h"].isnull()].iloc[:5, 5:20]

In [ ]:
df["market_cap_change_24h"].fillna(0, inplace=True)
df["market_cap_change_percentage_24h"].fillna(0, inplace=True)

## Verificamos la columna `total_supply`


In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
df[df["total_supply"].isnull()].iloc[:5, 15:20]

Supondremos que el `circulating_supply` es el mismo que `total_supply`, ya que en algunas monedas es comun esto

In [ ]:
df[df["circulating_supply"] != df["total_supply"]].iloc[:5, 15:20]

In [ ]:
df["total_supply"].fillna(df["circulating_supply"], inplace=True)

## Verificamos la columna `max_supply`

In [ ]:
find_columns_with_nulls(df)[0]

In [ ]:
df[df["max_supply"].isnull()].iloc[:5, 15:20]

Que `max_supply` sa nulo puede significar que la moneda no tiene limite de emision

In [ ]:
df["max_supply"].fillna(-1, inplace=True)

Una vez limpio los datos podemos observar los datos estadisticos de estos

In [ ]:
df.describe()

In [ ]:
df.info()

Guardamos el dataset limpio


In [ ]:
df.to_json("../../datasets/json/coins_clean.json", orient="records")